In [9]:
import newspaper
from newspaper import news_pool

urls = newspaper.popular_urls()

In [10]:
hot = newspaper.hot()

In [11]:
# build news sources 
papers = [newspaper.build(x) for x in urls]

In [5]:
[x for x in papers[0].category_urls()]

['https://www.huffingtonpost.gr',
 'https://www.huffingtonpost.kr',
 'http://www.huffingtonpost.com',
 'https://img.huffingtonpost.com',
 'http://www.huffingtonpost.com/entertainment',
 'https://www.huffingtonpost.jp',
 'http://www.huffingtonpost.com/life',
 'https://www.huffingtonpost.es',
 'https://www.huffingtonpost.fr',
 'https://www.huffingtonpost.co.uk',
 'http://www.huffingtonpost.com/voices',
 'https://www.huffingtonpost.it']

In [6]:
def filter_url(u:str)->str:
    if u.startswith("https://"):
        u = u[8:]
    elif u.startswith("http://"):
        u= u[7:]
    if u.startswith("www."):
        u = u[4:]
    if u.endswith(".com"):
        u = u[:-4]
    return u

def diff_to_base(durl: str, burl: str):
    filtered_durl = filter_url(durl)
    filtered_burl = filter_url(burl)
    
    if filtered_durl.startswith(filtered_burl):
        diff = filtered_durl[len(filtered_burl):]
        if diff.startswith('.'):
            diff = diff[1:]
        elif diff.startswith('/'):
            diff = diff[1:]
        if diff == "":
            return "base"
        return diff
    return "base"
    

def format_url(u:str)-> str:
    u = u.replace(".", "-")
    u = u.replace("/", ":")

for paper in papers[0].category_urls():
    print(diff_to_base(paper, papers[0].url))

gr
kr
base
base
com/entertainment
jp
com/life
es
fr
co.uk
com/voices
it


In [26]:
import time
t1 = time.time()
news_pool.set(papers, threads_per_source=2)
news_pool.join()
t2 = time.time()
print(int(t2-t1))



Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


4396


In [7]:
for paper in papers:
    print(paper.size())

284
386
16
0
0
129
423
0
182
14
153
0
22
0
0
144
31
0
119
3
70
0
49
41
0
9
37
2
25
10
56
2
42
42
0
47
0
0
0
40
0
1
0
2
3
0
82
6
0
4
12
10
0
72
30
62
1
18
576
79
8
0
1
5
324
15
78
144
954
0
35
79
20
134
9
7
0
35
22
4
0
0
65
0
12
23
22
272
34
0
43
2
46
0
83
0
15
0
56
62
13
346
0
0
22
48
29
0
165
59
36
0
0
79
48
21
19
83
26
7
57
119
0
173
16
138
0
19
0
0
27
0
6
0
11
131
31
175
2
45
2
63
0
0
0
6
92
13
224
0
0
7
31
32
188
6
298
14
0
60
49
5
0
106
0
113
0
12
0
35
139
22
33
6
0
77
84
1
83
1
1443
0
0
174
0
0
12
147
4
45
0
40
68
0
13
20
94
0
69
56
94
427
47
6
0
85
10
9
0
0
5
67
6
0
0
44
49
42
14
6
23
6
0
0
7
0
27
0
141
57
5
36
27
0
0
171
13
0
0
0
36
0


In [8]:
import nltk
nltk.download('punkt')

articles = [paper.articles for paper in papers]
n = 0
an = 0
for aa in articles:
    for a in aa:
        try:
            a.parse()
            a.nlp()
            an += 1
        except LookupError as E:
            n += 1
        except newspaper.ArticleException as E:
            n+= 1
        
        #now we have a.keywords, a.summary
print(a)
print(n)
t3 = time.time()
print(int(t3-t2))


13343


[nltk_data] Downloading package punkt to /Users/a2/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


NameError: name 'time' is not defined

In [ ]:
news_root_path = "/Users/a2/code/fin/trade/data/news"


In [ ]:
import json, os
from news import Article
from datetime import datetime
news_root_path = "/Users/a2/code/fin/trade/data/news"

def date_to_ddmmjj():
    # Datum einlesen
    date_obj = datetime.today()
    # In gewünschtes Format umwandeln
    formatted_date = date_obj.strftime('%d%m%y')
    return formatted_date
def sort_strings_by_length(strings):
    """
    Sort a list of strings by length, longest string first.

    :param strings: List of strings to be sorted
    :return: List of strings sorted by length
    """
    return sorted(strings, key=len, reverse=True)

today_path = os.path.join(news_root_path, date_to_ddmmjj())


### initialize folders for news dump
if not os.path.isdir(today_path):
    os.mkdir(today_path)
# papers[n][articles[m]]
    base_url_to_folder_name = {}
    cat_urls_to_folder_names = []
    for paper in papers:
        folder_name = format_url(filter_url(paper.url))
        base_url_to_folder_name[paper.url] = folder_name
        paper_folder = os.path.join(today_path, folder_name)
        os.mkdir(paper_folder)
        cat_urls_to_folder_names.append({})
        for  u in paper.category_urls():
            d = format_url(diff_to_base(u, paper.url))
            cat_urls_to_folder_names[-1][u] = d
            os.mkdir(os.path.join(paper_folder, d))
        

#### newsdump
for p, ip in enumerate(articles):
    paper_url = papers[ip].url
    pat = os.path.join(today_path, base_url_to_folder_name[paper_url])
    category_urls = sort_strings_by_length(p.category_urls())
    for a, ia in enumerate(p):
        article_url = a.url
        for cat_url in category_urls:
            if article_url.startswith(cat_url):
                pat = os.path.join(pat, cat_urls_to_folder_names[ip][cat_url])
                article_obj = Article(base_url_to_folder_name[paper_url], article_url, str(int(a.publish_date.timestamp())), a.authors, a.title, a.text, a.keywords, a.summary)
                with open(pat+"/A_"+ hash(a.summary), "w") as f:
                    f.write(article_obj.get_article())
                with open(pat+"/N_"+ hash(a.summary), "w") as f:
                    f.write(article_obj.get_nlp())
                break
        
    
    

TypeError: join() argument must be str, bytes, or os.PathLike object, not 'NoneType'